In [1]:
from qiskit_braket_provider import AWSBraketProvider
from braket.aws import AwsSession

aws_session = AwsSession(default_bucket="amazon-braket-us-west-1-lukasvoss")
AWSBraketProvider().backends()

[BraketBackend[Aria 1],
 BraketBackend[Aria 2],
 BraketBackend[Aspen-M-3],
 BraketBackend[Forte 1],
 BraketBackend[Harmony],
 BraketBackend[Lucy],
 BraketBackend[SV1],
 BraketBackend[dm1]]

In [2]:
import numpy as np
import sys
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"
module_path = os.path.abspath(os.path.join('/Users/lukasvoss/Documents/Master Wirtschaftsphysik/Masterarbeit Yale-NUS CQT/Quantum_Optimal_Control'))
if module_path not in sys.path:
    sys.path.append(module_path)

from template_configurations import gate_q_env_config
from quantumenvironment import QuantumEnvironment
from gymnasium.wrappers import RescaleAction, ClipAction

import logging
logging.basicConfig(
    level=logging.WARNING,
    format="%(asctime)s INFO %(message)s", # hardcoded INFO level
    datefmt="%Y-%m-%d %H:%M:%S",
    stream=sys.stdout,
)

/Users/lukasvoss/anaconda3/envs/braket/lib/python3.10/site-packages/qiskit_dynamics/dispatch/backends/jax.py:34: UserWarning: The functionality in the perturbation module of Qiskit Dynamics requires a JAX version <= 0.4.6, due to a bug in JAX versions > 0.4.6. For versions 0.4.4, 0.4.5, and 0.4.6, using the perturbation module functionality requires setting os.environ['JAX_JIT_PJIT_API_MERGE'] = '0' before importing JAX or Dynamics.
  warnings.warn(
I0000 00:00:1707965043.525374       1 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.


Circuit context
global phase: π
     ┌──────────┐┌────────┐                    ┌──────────┐┌────────┐
q_0: ┤ Ry(-π/2) ├┤ Rz(-π) ├──■────■────■────■──┤ Ry(-π/2) ├┤ Rz(-π) ├
     └──────────┘└────────┘┌─┴─┐  │    │    │  └──────────┘└────────┘
q_1: ──────────────────────┤ X ├──┼────┼────┼────────────────────────
                           └───┘┌─┴─┐  │    │                        
q_2: ───────────────────────────┤ X ├──┼────┼────────────────────────
                                └───┘┌─┴─┐  │                        
q_3: ────────────────────────────────┤ X ├──┼────────────────────────
                                     └───┘┌─┴─┐                      
q_4: ─────────────────────────────────────┤ X ├──────────────────────
                                          └───┘                      
Starting Rabi experiment for qubit 0...
Rabi experiment for qubit 0 done.
Starting Drag experiment for qubit 0...
Drag experiments done for qubit 0 done.
Starting Rabi experiment for qubit 1...
Rabi

/Users/lukasvoss/anaconda3/envs/braket/lib/python3.10/site-packages/qiskit_experiments/calibration_management/calibrations.py:1391: UserWarning: Schedules are only saved in text format. They cannot be re-loaded.
  warnings.warn("Schedules are only saved in text format. They cannot be re-loaded.")


In [3]:
# Define the original action space
print('Initial lower bounds:', gate_q_env_config.action_space.low)
print('Initial upper bounds:', gate_q_env_config.action_space.high)

q_env = QuantumEnvironment(gate_q_env_config)

# Apply the RescaleAction wrapper
q_env = ClipAction(q_env)
q_env = RescaleAction(q_env, min_action=-1.0, max_action=1.0)

# Confirm the rescale box dimensions
print('Rescaled lower bounds:', q_env.action_space.low)
print('Rescaled upper bounds:', q_env.action_space.high)

Initial lower bounds: [-3.14 -3.14 -3.14 -3.14 -3.14 -3.14 -3.14]
Initial upper bounds: [3.14 3.14 3.14 3.14 3.14 3.14 3.14]
None


TypeError: unsupported operand type(s) for *: 'NoneType' and 'int'

In [ ]:
gate_name = q_env.target['gate'].name
gate_name

In [ ]:
print(q_env.backend)

In [ ]:
from helper_functions import load_agent_from_yaml_file
agent_config  = load_agent_from_yaml_file(file_path='/Users/lukasvoss/Documents/Master Wirtschaftsphysik/Masterarbeit Yale-NUS CQT/Quantum_Optimal_Control/template_configurations/agent_config.yaml')

In [ ]:
agent_config

In [ ]:
from ppo import make_train_ppo

ppo_agent = make_train_ppo(agent_config, q_env)

In [ ]:
num_updates = 400

In [ ]:
training_results = ppo_agent(total_updates=num_updates, print_debug=True, num_prints=40)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import time

plt.style.use('ggplot')

In [ ]:
avg_reward = training_results['avg_reward']
std_actions = training_results['std_actions']
fidelities = training_results['fidelities']

In [ ]:
job_name = f'{gate_name}-gate-calibration-{int(time.time())}-max-fidelity-{round(max(fidelities), 5)}.pickle'

In [ ]:
job_name = f'{gate_name}-gate-calibration-{int(time.time())}-max-fidelity-{max(fidelities):4%}.pickle'

with open(job_name, 'wb') as handle:
    pickle.dump(training_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
print(f'Final Gate Fidelity: {fidelities[-1]:.4%}')
print(f'\nMax Gate Fidelity: {max(fidelities):.4%}')

In [ ]:
reward_history = np.array(q_env.reward_history)
mean_rewards = np.mean(reward_history, axis=-1)
max_mean = int(np.max(mean_rewards) * 1e4) / 1e4

fidelity = np.array(q_env.avg_fidelity_history)
mean_fidelity = np.mean(fidelity, axis=-1)
max_fidelity = int(np.max(mean_rewards) * 1e4) / 1e4

plt.plot(mean_rewards, label=f'Mean Batch Rewards, max: {max_mean}')
plt.plot(q_env.avg_fidelity_history, label=f'Mean Batch Gate Fidelity, max: {max_fidelity}')
plt.xlabel('Updates')
plt.title('CX Learning Curve')
plt.legend()
plt.show()

In [ ]:
std_actions_componentwise = list(zip(*std_actions))

for ind, param in enumerate(std_actions_componentwise):
    plt.plot(np.arange(1, num_updates+1), param, label=r'std $\theta_{}$'.format(ind+1))

plt.xlabel('update step')
plt.legend()

In [ ]:
print('Final action vector:\n', training_results['action_vector'])